<a href="https://colab.research.google.com/github/chetools/CHE4071_Spring2025/blob/main/ExothermicPFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

In [64]:
N=20
q = 100 #L/min
totV = 100 #L
V = totV/N
rho = 1000 #g/L
C = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750 #K
k0 = 7.2e10 #1/min
UA = 5e4/N #J/(min K)
Tc0 = 300 #K
Tc = 300 #K
ca0 = np.full(N, 0.5) #mol/L
T0 = np.full(N, 338.5) #K
vec0 = np.r_[ca0, T0]

In [65]:
def cin(t):
    return 1.

def Tin(t):
    return 338.5

In [79]:
def rhs(t, vec):
    ca, T = np.split(vec, 2)
    k = k0*np.exp(-ER/T)

    dc0 = q/V*(cin(t) - ca[0])  - k[0]*ca[0]
    dc1_n = q/V*(ca[:-1]- ca[1:]) - k[1:]*ca[1:]

    dT0 = q/V * (Tin(t) - T[0]) + negHr*k[0]*ca[0]/(rho*C) - UA/(rho*V*C)*(T[0]-Tc)
    dT1_n = q/V * (T[:-1] - T[1:]) + negHr*k[1:]*ca[1:]/(rho*C) - UA/(rho*V*C)*(T[1:]-Tc)


    return np.r_[dc0, dc1_n, dT0, dT1_n]

In [71]:
tend = 10 #min
res=solve_ivp(rhs, (0,tend), vec0, method='Radau', dense_output=True)

In [72]:
tplot = np.linspace(0,tend,101)
cplot, Tplot = np.split(res.sol(tplot), 2, axis=0)

In [77]:
vec_ss0=res.sol(tend)

In [74]:
fig=make_subplots(rows=1, cols=2)

for i in range(N):
    fig.add_scatter(x=tplot, y=cplot[i,:], row=1, col=1, name=f'c{i}')
    fig.add_scatter(x=tplot, y=Tplot[i,:], row=1, col=2, name=f'T{i}')


fig.update_layout(width=800, height=400, template='plotly_dark')

In [81]:

def Tin_ramp(t):
    if t>2:
        return 340
    return (340-338.5)*t/2 + 338.5

In [82]:
def rhs_ramp(t, vec):
    ca, T = np.split(vec, 2)
    k = k0*np.exp(-ER/T)

    dc0 = q/V*(cin(t) - ca[0])  - k[0]*ca[0]
    dc1_n = q/V*(ca[:-1]- ca[1:]) - k[1:]*ca[1:]

    dT0 = q/V * (Tin_ramp(t) - T[0]) + negHr*k[0]*ca[0]/(rho*C) - UA/(rho*V*C)*(T[0]-Tc)
    dT1_n = q/V * (T[:-1] - T[1:]) + negHr*k[1:]*ca[1:]/(rho*C) - UA/(rho*V*C)*(T[1:]-Tc)


    return np.r_[dc0, dc1_n, dT0, dT1_n]

In [84]:
tend = 10 #min
res=solve_ivp(rhs_ramp, (0,tend), vec_ss0, method='Radau', dense_output=True)

In [85]:
tplot = np.linspace(0,tend,101)
cplot, Tplot = np.split(res.sol(tplot), 2, axis=0)

In [87]:
fig1=make_subplots(rows=1, cols=2)

for i in range(N):
    fig1.add_scatter(x=tplot, y=cplot[i,:], row=1, col=1, name=f'c{i}')
    fig1.add_scatter(x=tplot, y=Tplot[i,:], row=1, col=2, name=f'T{i}')


fig1.update_layout(width=800, height=400, template='plotly_dark')